- https://youtu.be/f1KUUz7v8g4?t=3570
這裡的regularization是啥 (0.5,0.5)比(0.1,0.9)好
假如weight平均分佈，這樣不就都一樣，應該是不想要weight都集中在某個畫面吧

- point network: 從input的vec裡面選出一個vec

- https://youtu.be/z0uOq2wEGcc?t=634
怎麼知道這樣的結構

- GRU裡面的dropout

# import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
import torch.utils.data.sampler as sampler
import torchvision
from torchvision import datasets, transforms

import numpy as np
import sys
import os
import random
import json


os.environ["CUDA_VISIBLE_DEVICES"]="3"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 判斷是用 CPU 還是 GPU 執行運算


# utils

## options

In [2]:
class Options(object):
      def __init__(self):
        self.batch_size = 60
        self.emb_dim = 256
        self.hid_dim = 512
        self.n_layers = 3
        self.dropout = 0.5
        self.learning_rate = 0.00005
        self.seq_len = 50              # 最後輸出句子的最大長度
        self.num_steps = 12000                # 總訓練次數
        self.store_steps = 300                # 訓練多少次後須儲存模型
        self.summary_steps = 300              # 訓練多少次後須檢驗是否有overfitting
        self.load_model = False               # 是否需載入模型
        self.store_model_path = "./ckpt"      # 儲存模型的位置
        self.load_model_path = None           # 載入模型的位置 e.g. "./ckpt/model_{step}" 
        self.data_dir = '/workdir/home/feynman52/NTU-ML2020/hw8-Seq2seq/datasets/cmn-eng/'          # 資料存放的位置
        self.attention = False                # 是否使用 Attention Mechanism

In [ ]:
options = Options()
options.num_steps

## int2str

In [3]:
def intSen2strSen(intSen, int2word):
    strSen = []
    for x in intSen:
        word = int2word[str(int(x))]
        if word == '<EOS>': break
        strSen.append(word)
    return strSen

def intSens2strSens(intSens, int2word):
    strSens = [intSen2strSen(intSen, int2word) for intSen in intSens] 
    return strSens

In [ ]:
i = 5
print(intSen2strSen(x[i], train_set.int2word_en))
print(intSen2strSen(y[i], train_set.int2word_cn))

# data

## example

In [ ]:
class LabelTransform(object):
    def __init__(self, size, pad):
        self.size = size
        self.pad = pad

    def __call__(self, label):
        label = np.pad(label, (0, (self.size - label.shape[0])), mode='constant', constant_values=self.pad)
        return label

In [ ]:
import re
import json

class EN2CNDataset(data.Dataset):
    def __init__(self, root, max_output_len, set_name):
        self.root = root

        self.word2int_cn, self.int2word_cn = self.get_dictionary('cn')
        self.word2int_en, self.int2word_en = self.get_dictionary('en')

        # 載入資料
        self.data = []
        with open(os.path.join(self.root, f'{set_name}.txt'), "r") as f:
            for line in f:
                self.data.append(line)
        print (f'{set_name} dataset size: {len(self.data)}')

        self.cn_vocab_size = len(self.word2int_cn)
        self.en_vocab_size = len(self.word2int_en)
        self.transform = LabelTransform(max_output_len, self.word2int_en['<PAD>'])

    def get_dictionary(self, language):
        # 載入字典
        with open(os.path.join(self.root, f'word2int_{language}.json'), "r") as f:
            word2int = json.load(f)
        with open(os.path.join(self.root, f'int2word_{language}.json'), "r") as f:
            int2word = json.load(f)
            return word2int, int2word

    def __len__(self):
        return len(self.data)

    def __getitem__(self, Index):
        print('#'*50)
        # 先將中英文分開
        sentences = self.data[Index]
        print(sentences)
        sentences = re.split('[\t\n]', sentences)
        print(sentences)
        sentences = list(filter(None, sentences))
        print(sentences)
        #print (sentences)
        assert len(sentences) == 2

        # 預備特殊字元
        BOS = self.word2int_en['<BOS>']
        EOS = self.word2int_en['<EOS>']
        UNK = self.word2int_en['<UNK>']

        print('#'*50)
        # 在開頭添加 <BOS>，在結尾添加 <EOS> ，不在字典的 subword (詞) 用 <UNK> 取代
        en, cn = [BOS], [BOS]
        # 將句子拆解為 subword 並轉為整數
        sentence = re.split(' ', sentences[0])
        print(sentence)
        sentence = list(filter(None, sentence)) # https://stackoverflow.com/questions/3845423/remove-empty-strings-from-a-list-of-strings
        print(sentence)
        #print (f'en: {sentence}')
        for word in sentence:
            en.append(self.word2int_en.get(word, UNK))
        en.append(EOS)
        print(en)

        print('#'*50)
        # 將句子拆解為單詞並轉為整數
        # e.g. < BOS >, we, are, friends, < EOS > --> 1, 28, 29, 205, 2
        sentence = re.split(' ', sentences[1])
        print(sentence)
        sentence = list(filter(None, sentence))
        print(sentence)
        #print (f'cn: {sentence}')
        for word in sentence:
            cn.append(self.word2int_cn.get(word, UNK))
        cn.append(EOS)
        print(cn)

        en, cn = np.asarray(en), np.asarray(cn)

        # 用 <PAD> 將句子補到相同長度
        en, cn = self.transform(en), self.transform(cn)
        en, cn = torch.LongTensor(en), torch.LongTensor(cn)

        return en, cn

## check file

In [ ]:
root = '/workdir/home/feynman52/NTU-ML2020/hw8-Seq2seq/datasets/cmn-eng'
max_output_len = 20
set_name = 'training'
eN2CNDataset = EN2CNDataset(root, max_output_len, set_name)

In [ ]:
list(filter(None, ['what', 'exactly', 'does', 'tom', 'do', '?', '']))

In [ ]:
# encoder biriectional => padding在後面沒差!!!

In [ ]:
(eN2CNDataset[10])

In [ ]:
eN2CNDataset.data

In [ ]:
with open('/workdir/home/feynman52/NTU-ML2020/hw8-Seq2seq/datasets/cmn-eng/int2word_cn.json', "r") as f:
    int2word_cn = json.load(f)
    
int2word_cn

In [ ]:
with open('/workdir/home/feynman52/NTU-ML2020/hw8-Seq2seq/datasets/cmn-eng/word2int_cn.json', "r") as f:
    word2int = json.load(f)
    
word2int

In [ ]:
with open('/workdir/home/feynman52/NTU-ML2020/hw8-Seq2seq/datasets/cmn-eng/training.txt', 'r') as f:
    content = f.readlines()
    
content = [x.strip() for x in content]     
content

## dataset

In [ ]:
cn = [1,2,3,4,5]
cn += ['<PAD>'] * (3 - len(cn))
print(cn)

cn = [1,2,3,4,5]
cn += ['<PAD>'] * (8 - len(cn))
print(cn)

In [ ]:
# np.pad([4,5,6], (0, -2), mode='constant', constant_values=(8))

In [4]:
class Seq2seqDataset(data.Dataset):
    def __init__(self, data_dir=None, which_dataset='testing.txt', seq_len=20):
        self.data_dir = data_dir
        self.which_dataset = which_dataset
        self.seq_len = seq_len
        self.setup()
        
    def setup(self):
        self.setup_dict()
        self.setup_data()
    
    def setup_dict(self):
        self.int2word_cn = self.load_json('int2word_cn.json') 
        self.int2word_en = self.load_json('int2word_en.json') 
        
        self.word2int_cn = self.load_json('word2int_cn.json') 
        self.word2int_en = self.load_json('word2int_en.json') 
        
        self.cn_dict_size = len(self.int2word_cn)
        self.en_dict_size = len(self.int2word_en)
    
    def load_json(self, path):
        with open(os.path.join(self.data_dir, path), "r") as f:
            j = json.load(f)
        return j
    
    def setup_data(self):
        with open(os.path.join(self.data_dir, self.which_dataset), "r") as f:
            self.data = f.readlines()
        self.sentences_en = []
        self.sentences_cn = []
        for en_cn in self.data:
            en, cn = en_cn.replace(' \n', '').split(' \t')
            en = self.process_word2int(en, self.word2int_en)
            cn = self.process_word2int(cn, self.word2int_cn)
            self.sentences_en.append(en)
            self.sentences_cn.append(cn)
            
    def process_word2int(self, cn, word2int_cn):
        cn = cn.split(' ') 
        cn = ['<BOS>'] + cn + ['<EOS>']
        cn += ['<PAD>'] * (self.seq_len - len(cn)) # 因為encoder是bi-direction, 所以padding在前面後面沒差
        cn = cn[:self.seq_len] # 假如字超過seq_len，取前一部份的長度
        cn = [word2int_cn[x] if x in word2int_cn else word2int_cn['<UNK>'] for x in cn]
        cn = torch.LongTensor(cn)
        return cn
    
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, index):
        #print(self.data[index])
        sentence_en = self.sentences_en[index]
        sentence_cn = self.sentences_cn[index]
        return sentence_en, sentence_cn
        

In [ ]:
data_dir = '/workdir/home/feynman52/NTU-ML2020/hw8-Seq2seq/datasets/cmn-eng/'
train_set = Seq2seqDataset(data_dir, which_dataset='training.txt')
valid_set = Seq2seqDataset(data_dir, which_dataset='validation.txt')
test_set = Seq2seqDataset(data_dir, which_dataset='testing.txt')


In [ ]:
# en_cn = seq2seqDataset.data[0]
# en, cn = en_cn.replace(' \n', '').split(' \t')

# word2int_cn = seq2seqDataset.word2int_cn
# int2word_cn = seq2seqDataset.int2word_cn
# cn = cn.split(' ') 
# print(cn)
# cn = ['<BOS>'] + cn + ['<EOS>']
# cn += ['<PAD>'] * (seq2seqDataset.seq_len - len(cn))
# cn = [word2int_cn[x] if x in word2int_cn else word2int_cn['<UNK>'] for x in cn]
# print(cn)
# cn_reverse = [int2word_cn[str(x)] for x in cn]
# print(len(cn_reverse))
# cn_reverse

In [ ]:
len(train_set), len(valid_set), len(test_set), 

## dataloader

In [ ]:
batch_size = 50
train_loader = data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = data.DataLoader(valid_set, batch_size=batch_size, shuffle=False)
test_loader = data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
len(train_loader), len(valid_loader), len(test_loader), 

In [ ]:
x, y = next(iter(train_loader))
print(x.shape, y.shape)
print(x[0], y[0])

In [ ]:
#? 為啥test data也有y
x, y = next(iter(test_loader))
print(x.shape, y.shape)
print(x[0], y[0])

# model

## example

In [ ]:
class Encoder(nn.Module):
    def __init__(self, en_vocab_size, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(en_vocab_size, emb_dim)
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input):
        # input = [batch size, sequence len, vocab size]
        embedding = self.embedding(input)
        output, h_n = self.rnn(self.dropout(embedding))
        # output of shape (batch, seq_len, num_directions * hidden_size)
        # h_n of shape (batch, num_layers * num_directions, hidden_size)

        return output, h_n


In [ ]:
class Decoder(nn.Module):
    def __init__(self, cn_vocab_size, emb_dim, hid_dim, n_layers, dropout, isatt):
        super().__init__()
        self.cn_vocab_size = cn_vocab_size
        self.hid_dim = hid_dim * 2
        self.n_layers = n_layers
        self.embedding = nn.Embedding(cn_vocab_size, config.emb_dim)
        self.isatt = isatt
        self.attention = Attention(hid_dim)
        
        # 如果使用 Attention Mechanism 會使得輸入維度變化，請在這裡修改
        # e.g. Attention 接在輸入後面會使得維度變化，所以輸入維度改為
        # self.input_dim = emb_dim + hid_dim * 2 if isatt else emb_dim
        self.input_dim = emb_dim
        
        self.rnn = nn.GRU(self.input_dim, self.hid_dim, self.n_layers, dropout = dropout, batch_first=True)
        self.embedding2vocab1 = nn.Linear(self.hid_dim, self.hid_dim * 2)
        self.embedding2vocab2 = nn.Linear(self.hid_dim * 2, self.hid_dim * 4)
        self.embedding2vocab3 = nn.Linear(self.hid_dim * 4, self.cn_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        # input = [batch size, vocab size] ### (50, 20) 這邊應該寫錯了，這裡是 (batch, seq_len)
        # hidden = [batch size, n layers * directions, hid dim]
        # Decoder 只會是單向，所以 directions=1
        input = input.unsqueeze(1)
        embedded = self.dropout(self.embedding(input))
        # embedded = [batch size, 1, emb dim]
        
        if self.isatt:
            attn = self.attention(encoder_outputs, hidden)
            # TODO: 在這裡決定如何使用 Attention，e.g. 相加 或是 接在後面， 請注意維度變化
            
        output, hidden = self.rnn(embedded, hidden)
        # output = [batch size, 1, hid dim]
        # hidden = [batch size, num_layers, hid dim]

        # 將 RNN 的輸出轉為每個詞出現的機率
        output = self.embedding2vocab1(output.squeeze(1))
        output = self.embedding2vocab2(output)
        prediction = self.embedding2vocab3(output)
        # prediction = [batch size, cn_vocab_size]
        return prediction, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim

    def forward(self, encoder_outputs, decoder_hidden):
        # encoder_outputs = [batch size, sequence len, hid dim * directions]
        # decoder_hidden = [num_layers, batch size, hid dim]
        # 一般來說是取 Encoder 最後一層的 hidden state 來做 attention
        ########
        # TODO #
        ########
        attention=None

        return attention

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert encoder.n_layers == decoder.n_layers, \
                "Encoder and decoder must have equal number of layers!"

    def forward(self, input, target, teacher_forcing_ratio):
        # input  = [batch size, input len, vocab size]
        # target = [batch size, target len, vocab size]
        # teacher_forcing_ratio 是有多少機率使用正確答案來訓練
        batch_size = target.shape[0]
        target_len = target.shape[1]
        vocab_size = self.decoder.cn_vocab_size

        # 準備一個儲存空間來儲存輸出
        outputs = torch.zeros(batch_size, target_len, vocab_size).to(self.device)
        
        # 將輸入放入 Encoder
        encoder_outputs, hidden = self.encoder(input)
        
        # Encoder 最後的隱藏層(hidden state) 用來初始化 Decoder
        # encoder_outputs 主要是使用在 Attention
        # 因為 Encoder 是雙向的RNN，所以需要將同一層兩個方向的 hidden state 接在一起
        # hidden =  [num_layers * directions, batch size  , hid dim]  --> [num_layers, directions, batch size  , hid dim]
        hidden = hidden.view(self.encoder.n_layers, 2, batch_size, -1)
        hidden = torch.cat((hidden[:, -2, :, :], hidden[:, -1, :, :]), dim=2) ###
        
        # 取得 <BOS> token
        input = target[:, 0] ###
        preds = []
        for t in range(1, target_len):
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            outputs[:, t] = output ### [batch, cn_vocab_size]
            
            # 決定是否用正確答案來做訓練
            teacher_force = (random.random() <= teacher_forcing_ratio)
            
            # 取出機率最大的單詞
            top1 = output.argmax(1)
            
            # 如果是 teacher force 則用正解訓練，反之用自己預測的單詞做預測
            input = target[:, t] if teacher_force and t < target_len else top1
            preds.append(top1.unsqueeze(1))
        preds = torch.cat(preds, 1)
        return outputs, preds

    def inference(self, input, target):
        ########
        # TODO #
        ########
        # 在這裡實施 Beam Search
        # 此函式的 batch size = 1  
        # input  = [batch size, input len, vocab size]
        # target = [batch size, target len, vocab size]
        batch_size = input.shape[0]
        input_len = input.shape[1]        # 取得最大字數
        vocab_size = self.decoder.cn_vocab_size

        # 準備一個儲存空間來儲存輸出
        outputs = torch.zeros(batch_size, input_len, vocab_size).to(self.device)
        # 將輸入放入 Encoder
        encoder_outputs, hidden = self.encoder(input)
        # Encoder 最後的隱藏層(hidden state) 用來初始化 Decoder
        # encoder_outputs 主要是使用在 Attention
        # 因為 Encoder 是雙向的RNN，所以需要將同一層兩個方向的 hidden state 接在一起
        # hidden =  [num_layers * directions, batch size  , hid dim]  --> [num_layers, directions, batch size  , hid dim]
        hidden = hidden.view(self.encoder.n_layers, 2, batch_size, -1)
        hidden = torch.cat((hidden[:, -2, :, :], hidden[:, -1, :, :]), dim=2)
        # 取的 <BOS> token
        input = target[:, 0]
        preds = []
        for t in range(1, input_len):
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            # 將預測結果存起來
            outputs[:, t] = output
            # 取出機率最大的單詞
            top1 = output.argmax(1)
            input = top1
            preds.append(top1.unsqueeze(1))
        preds = torch.cat(preds, 1)
        return outputs, preds


## EncoderModel

In [6]:
class EncoderModel(nn.Module):
    def __init__(self, en_dict_size=10000, dropout=0.2, enc_h_dim=10, embedding_dim=20, num_layers=5):
        super().__init__()
        self.en_dict_size = en_dict_size
        self.enc_h_dim = enc_h_dim
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(num_embeddings=en_dict_size, 
                                      embedding_dim=embedding_dim)
        
        self.gru = nn.GRU(input_size=embedding_dim, 
                          hidden_size=enc_h_dim, 
                          num_layers=num_layers, 
                          bias=True, 
                          batch_first=True, 
                          dropout=dropout, 
                          bidirectional=True)
        
    def forward(self, enc_input): 
        egru_input = self.embedding(enc_input) # (batch, seq_len) => (batch, seq_len, embedding_dim)
        
        egru_output, egru_hidden = self.gru(egru_input)
        # egru_output = (batch, seq_len, num_directions * hidden_size) = (batch, seq_len, 2 * enc_h_dim)
        # egru_hidden = (num_layers * num_directions, batch, hidden_size) = (num_layers * 2, batch, enc_h_dim)
        
        return egru_output, egru_hidden
    
    

## AttentionModel

In [5]:
class AttentionModel(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim

    def forward(self, dgru_hidden, egru_output):
        # egru_output = (batch, seq_len, 2*enc_h_dim)
        # dgru_hidden = (num_layers, batch, enc_h_dim*2)
        
        attention = None # (batch, 2*enc_h_dim) #?

        return attention # (batch, 1, 2*enc_h_dim)

## DecoderModel

In [7]:
class DecoderModel(nn.Module):
    def __init__(self, cn_dict_size=10000, dropout=0.2, enc_h_dim=10, embedding_dim=20, num_layers=5):
        super().__init__() 
        self.dec_h_dim = enc_h_dim*2
        self.cn_dict_size = cn_dict_size
        self.enc_h_dim = enc_h_dim
        self.num_layers = num_layers
        self.dgru_input_dim = embedding_dim +  enc_h_dim*2 #?
        
        self.embedding = nn.Embedding(num_embeddings=cn_dict_size, 
                                      embedding_dim=embedding_dim)
        
        self.gru = nn.GRU(input_size=self.dgru_input_dim, 
                          hidden_size=self.dec_h_dim, 
                          num_layers=num_layers, 
                          bias=True, 
                          batch_first=True, 
                          dropout=dropout, 
                          bidirectional=False)
        
        self.dropout = nn.Dropout(dropout)
        self.dnn1 = nn.Linear(self.dec_h_dim, self.dec_h_dim * 2)
        self.dnn2 = nn.Linear(self.dec_h_dim * 2, self.dec_h_dim * 4)
        self.dnn3 = nn.Linear(self.dec_h_dim * 4, cn_dict_size)
        
    def forward(self, dec_input, dgru_hidden, egru_output):         
        x = self.embedding(dec_input) # (batch, 1) => (batch, 1, emb_dim)
        x = self.dropout(x) # (batch, 1, emb_dim) 
        
        attention = self.AttentionModel() # (batch, 1, 2*enc_h_dim)
        dgru_input = concat([x, attention], dim=2) # (batch, 1, emb_dim + 2*enc_h_dim)
        
        dgru_output, dgru_hidden = self.gru(dgru_input, dgru_hidden) 
        # dgru_output = (batch, seq_len, num_directions * hidden_size) = (batch, 1, dec_h_dim)
        # dgru_hidden = (num_layers * num_directions, batch, hidden_size) = (num_layers, batch, dec_h_dim)
        
        dec_output = dgru_output.squeeze(1) # (batch, dec_h_dim)
        dec_output = self.dnn1(dec_output)
        dec_output = self.dnn2(dec_output)
        dec_output = self.dnn3(dec_output) # (batch, cn_dict_size)
        return dec_output, dgru_hidden
    

In [ ]:
# x = torch.zeros(50, 20)
# print(x.shape)
# x = x[:, 10:11]
# print(x.shape)

In [ ]:
# x = torch.zeros(50, 20)
# print(x.shape)
# x = x[:, 10]
# print(x.shape)
# x = x.unsqueeze(1)
# print(x.shape)
# x = x.squeeze(1)
# print(x.shape)

In [ ]:
# x = torch.zeros(50, 1, 15)
# print(x.shape)
# x = x.squeeze(1)
# print(x.shape)

## Seq2seqModel

In [8]:
class Seq2seqModel(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, enc_input, target, teacher_forcing_ratio):
        batch_size = enc_input.shape[0]
        seq_len = enc_input.shape[1]
        
        egru_output, egru_hidden = self.encoder(enc_input)
        # enc_input = (batch, seq_len)
        # egru_output = (batch, seq_len, num_directions * hidden_size) = (batch, seq_len, 2 * enc_h_dim)
        # egru_hidden = (num_layers * num_directions, batch, hidden_size) = (num_layers * 2, batch, enc_h_dim)
        
        egru_hidden = egru_hidden.reshape(self.encoder.num_layers, 2, batch_size, self.encoder.enc_h_dim) 
        # batch_first不影響hidden的shape, (num_layers, 2, batch, enc_h_dim) 
        
        dgru_hidden = torch.cat([egru_hidden[:,0,:,:], egru_hidden[:,1,:,:]], dim=2) 
        # [(num_layers, batch, enc_h_dim)] => (num_layers, batch, enc_h_dim*2)
        
        dec_input = target[:, 0:1] # (batch, seq_len) => (batch, 1)
        
        one_hot_outputs = torch.zeros(batch_size, seq_len, self.decoder.cn_dict_size).to(self.device) 
        # (batch, seq_len, cn_dict_size)
        
        int_outputs = [] 
        for i in range(1, seq_len):
            dec_output, dgru_hidden = self.decoder(dec_input, dgru_hidden, egru_output)
            # dec_output = (batch, cn_dict_size)
            # dgru_hidden = (num_layers, batch, dec_h_dim)
            
            one_hot_outputs[:, i, :] = dec_output 
            
            top1 = one_hot_outputs.argmax(dim=1) # (batch, cn_dict_size) => (batch,)
            top1 = top1.unsqueeze(dim=1) # (batch, 1)
            int_outputs.append(top1) # [(batch, 1)]
            
            dec_input = target[:, i:i+1] if random.random() <= teacher_forcing_ratio else top1
            
        int_outputs = torch.cat(int_outputs, dim=1) # (batch, seq_len)
        
        return one_hot_outputs, int_outputs
    
    def inference(self, enc_input, target):
        batch_size = enc_input.shape[0]
        seq_len = enc_input.shape[1]
        
        enc_output, enc_hidden = self.encoder(enc_input)
        
        enc_hidden = enc_hidden.reshape(self.encoder.num_layers, 2, batch_size, self.encoder.enc_h_dim) 
        # batch_first不影響hidden的shape
        # (num_layers * num_directions, batch, hidden_size) => (num_layers, num_directions, batch, hidden_size) 
        
        enc_hidden = torch.cat([enc_hidden[:,0,:,:], enc_hidden[:,1,:,:]], dim=2) # [(num_layers, batch, hidden_size)] => (num_layers, batch, hidden_size*2)
        dec_hidden = enc_hidden
        
        #? 這裡應該不需要target才對，直接輸入BOS就好
        dec_input = target[:, 0:1] # (batch, seq_len) => (batch, 1) 
        dec_outputs = torch.zeros(batch_size, seq_len, self.decoder.cn_dict_size).to(self.device) # (batch, seq_len, cn_dict_size)
        seq_outputs = [] 
        for i in range(1, seq_len):
            dec_output, dec_hidden = self.decoder(dec_input, dec_hidden, enc_output)
            
            dec_outputs[:, i] = dec_output # (batch, cn_dict_size)
            
            top1 = dec_output.argmax(dim=1) # (batch, cn_dict_size) => (batch,)
            top1 = top1.unsqueeze(dim=1) # (batch, 1)
            seq_outputs.append(top1)
            
            dec_input = top1
            
        seq_outputs = torch.cat(seq_outputs, dim=1) # [(batch, 1)] => (batch, seq_len)
        
        return dec_outputs, seq_outputs
        
                
                
        

In [ ]:
# x = torch.zeros(50, 4)
# y = torch.zeros(23, 4)
# z = torch.cat([x, y], dim=0)
# z.shape

In [ ]:
# a = torch.randn(2,4)
# print(a, a.shape)
# b = torch.argmax(a, dim=1)
# print(b, b.shape)
# b = b.unsqueeze(dim=1)
# print(b, b.shape)

# train 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## data

In [ ]:
train_set = Seq2seqDataset(options.data_dir, which_dataset='training.txt')
valid_set = Seq2seqDataset(options.data_dir, which_dataset='validation.txt')
test_set = Seq2seqDataset(options.data_dir, which_dataset='testing.txt')
train_set.cn_dict_size, train_set.en_dict_size, 
len(train_set), len(valid_set)

In [ ]:
train_loader = data.DataLoader(train_set, batch_size=options.batch_size, shuffle=True)
valid_loader = data.DataLoader(valid_set, batch_size=options.batch_size, shuffle=False)
test_loader = data.DataLoader(test_set, batch_size=options.batch_size, shuffle=False)

In [ ]:
len(train_set), options.batch_size, len(train_loader)

## build model

In [ ]:
encoderModel = EncoderModel(en_dict_size=train_set.en_dict_size, 
                            dropout=options.dropout, 
                            enc_h_dim=options.hid_dim, 
                            embedding_dim=options.emb_dim, 
                            num_layers=options.n_layers)


In [ ]:
decoderModel = DecoderModel(cn_dict_size=train_set.cn_dict_size, 
                            dropout=options.dropout, 
                            enc_h_dim=options.hid_dim, 
                            embedding_dim=options.emb_dim, 
                            num_layers=options.n_layers)

In [ ]:
seq2seqModel = Seq2seqModel(encoderModel, decoderModel, device)
seq2seqModel = seq2seqModel.to(device)

In [ ]:
print(seq2seqModel)

In [9]:
def build_model(options, en_dict_size, cn_dict_size, device):
    encoderModel = EncoderModel(en_dict_size=en_dict_size, 
                                dropout=options.dropout, 
                                enc_h_dim=options.hid_dim, 
                                embedding_dim=options.emb_dim, 
                                num_layers=options.n_layers)

    decoderModel = DecoderModel(cn_dict_size=cn_dict_size, 
                                dropout=options.dropout, 
                                enc_h_dim=options.hid_dim, 
                                embedding_dim=options.emb_dim, 
                                num_layers=options.n_layers)

    seq2seqModel = Seq2seqModel(encoderModel, decoderModel, device)
    seq2seqModel = seq2seqModel.to(device)
    return seq2seqModel


In [ ]:
# seq2seqModel = build_model(options, train_set.en_dict_size, train_set.cn_dict_size, device)

## optimizer

In [ ]:
optimizer = torch.optim.Adam(seq2seqModel.parameters(), lr=options.learning_rate)

## loss function

In [ ]:
loss_function = nn.CrossEntropyLoss(ignore_index=0)

## train on batch

In [10]:
def schedule_sampling(epoch):
    # return f(epoch)
    return 1

In [ ]:
train_loss = 0.0
seq2seqModel.train()

for i, x_y in enumerate(train_loader):
    
    x, y = x_y[0].to(device), x_y[1].to(device)
    #print(i, x.shape)
    
    enc_input,  target = x, y
    teacher_forcing_ratio = schedule_sampling()
    dec_outputs, seq_outputs = seq2seqModel(enc_input, target, teacher_forcing_ratio)
    # dec_outputs = (batch_size, seq_len, dict_size)
    # seq_outputs = (batch_size, seq_len)
    
    dec_outputs = dec_outputs[:, 1:, :].reshape(-1, dec_outputs.shape[2]) # (batch_size, seq_len, dict_size) -> (batch_size*(seq_len-1), dict_size)
    target = target[:, 1:].reshape(-1) # (batch_size, seq_len) -> (batch_size*(seq_len-1))
    
    batch_loss = loss_function(dec_outputs, target)
    
    optimizer.zero_grad()
    batch_loss.backward()
    grad_norm = torch.nn.utils.clip_grad_norm_(seq2seqModel.parameters(), 1) 
    optimizer.step()
    
    train_loss += batch_loss.item()
    
train_loss /= len(train_set)


In [11]:
def train_model(seq2seqModel, train_loader, optimizer, loss_function, epoch, device):
    train_loss = 0.0
    seq2seqModel.train()
    teacher_forcing_ratio = schedule_sampling(epoch)
    
    for i, x_y in enumerate(train_loader):

        x, y = x_y[0].to(device), x_y[1].to(device)
        #print(i, x.shape)

        enc_input, target = x, y
        dec_outputs, seq_outputs = seq2seqModel(enc_input, target, teacher_forcing_ratio)
        # dec_outputs = (batch_size, seq_len, dict_size)
        # seq_outputs = (batch_size, seq_len)

        dec_outputs = dec_outputs[:, 1:, :].reshape(-1, dec_outputs.shape[2]) # (batch_size, seq_len, dict_size) -> (batch_size*(seq_len-1), dict_size)
        target = target[:, 1:].reshape(-1) # (batch_size, seq_len) -> (batch_size*(seq_len-1))

        batch_loss = loss_function(dec_outputs, target)

        optimizer.zero_grad()
        batch_loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(seq2seqModel.parameters(), 1) 
        optimizer.step()

        train_loss += batch_loss.item()

    train_loss /= len(train_loader.dataset)
    return seq2seqModel, train_loss, optimizer


## valid on batch

In [ ]:
for source, pred, target in zip([1,2,3], [4,5,6], [7,8,9]):
    print(source, pred, target)

In [ ]:
valid_loss = 0.0
seq2seqModel.eval()

results = []
for i, x_y in enumerate(valid_loader):
    
    x, y = x_y[0].to(device), x_y[1].to(device)
    #print(i, x.shape)
    
    enc_input,  target = x, y
    dec_outputs, seq_outputs = seq2seqModel.inference(enc_input, target)
    # dec_outputs = (batch_size, seq_len, dict_size)
    # seq_outputs = (batch_size, seq_len)
    
    dec_outputs = dec_outputs[:, 1:, :].reshape(-1, dec_outputs.shape[2]) # (batch_size, seq_len, dict_size) -> (batch_size*(seq_len-1), dict_size)
    target = target[:, 1:].reshape(-1) # (batch_size, seq_len) -> (batch_size*(seq_len-1))
    
    batch_loss = loss_function(dec_outputs, target)
    
    valid_loss += batch_loss.item()
    
    en_intSens, target_cn_intSens, predict_cn_intSens = x, y, seq_outputs
    
    en_strSens = intSens2strSens(en_intSens, valid_loader.dataset.int2word_en)
    target_cn_strSens = intSens2strSens(target_cn_intSens, valid_loader.dataset.int2word_cn)
    predict_cn_strSens = intSens2strSens(predict_cn_intSens, valid_loader.dataset.int2word_cn)
    for en_strSen, target_cn_strSen, predict_cn_strSen in zip(en_strSens, target_cn_strSens, predict_cn_strSens):
        results.append((en_strSen, target_cn_strSen, predict_cn_strSen))
    
    
valid_loss /= len(valid_set)


In [12]:
def test_model(seq2seqModel, valid_loader, loss_function, device):
    valid_loss = 0.0
    seq2seqModel.eval()

    results = []
    for i, x_y in enumerate(valid_loader):

        x, y = x_y[0].to(device), x_y[1].to(device)
        #print(i, x.shape)

        enc_input, target = x, y
        dec_outputs, seq_outputs = seq2seqModel.inference(enc_input, target)
        # dec_outputs = (batch_size, seq_len, dict_size)
        # seq_outputs = (batch_size, seq_len)

        dec_outputs = dec_outputs[:, 1:, :].reshape(-1, dec_outputs.shape[2]) # (batch_size, seq_len, dict_size) -> (batch_size*(seq_len-1), dict_size)
        target = target[:, 1:].reshape(-1) # (batch_size, seq_len) -> (batch_size*(seq_len-1))

        batch_loss = loss_function(dec_outputs, target)

        valid_loss += batch_loss.item()

        en_intSens, target_cn_intSens, predict_cn_intSens = x, y[:, 1:], seq_outputs

        en_strSens = intSens2strSens(en_intSens, valid_loader.dataset.int2word_en)
        target_cn_strSens = intSens2strSens(target_cn_intSens, valid_loader.dataset.int2word_cn)
        predict_cn_strSens = intSens2strSens(predict_cn_intSens, valid_loader.dataset.int2word_cn)
        
        for en_strSen, target_cn_strSen, predict_cn_strSen in zip(en_strSens, target_cn_strSens, predict_cn_strSens):
            result = (en_strSen, target_cn_strSen, predict_cn_strSen)
            results.append(result)

    valid_loss /= len(valid_loader.dataset)
    return valid_loss, results



## train on epoch

In [14]:
options = Options() 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
train_set = Seq2seqDataset(options.data_dir, 
                           which_dataset='training.txt', 
                           seq_len=options.seq_len)

valid_set = Seq2seqDataset(options.data_dir, 
                           which_dataset='validation.txt', 
                           seq_len=options.seq_len)

train_loader = data.DataLoader(train_set, batch_size=options.batch_size, shuffle=True)
valid_loader = data.DataLoader(valid_set, batch_size=options.batch_size, shuffle=False)

seq2seqModel = build_model(options, train_set.en_dict_size, train_set.cn_dict_size, device)
optimizer = torch.optim.Adam(seq2seqModel.parameters(), lr=options.learning_rate)
loss_function = nn.CrossEntropyLoss(ignore_index=0)

for epoch in range(5):
    # train
    seq2seqModel, train_loss, optimizer = train_model(seq2seqModel, 
                                                      train_loader, 
                                                      optimizer, 
                                                      loss_function, 
                                                      epoch, 
                                                      device)
    # valid
    valid_loss, results = test_model(seq2seqModel, 
                                     valid_loader, 
                                     loss_function, 
                                     device)
    print(train_loss, valid_loss)
    

0.07993608517116971 0.10909937477111817
0.06782084731260936 0.10667786693572998
0.06375954887602064 0.09921484375
0.06049883564313253 0.10013787269592285
0.057725648562113444 0.09684638500213623


In [ ]:
train_loss, valid_loss

In [ ]:
results[38]